# TensorFlow Data **Validation**

TensorFlow Data Validation (TFDV) is a library for exploring and validating  machine learning data. It is designed to be highly scalable and to work  well with TensorFlow and TensorFlow Extended (TFX). This notebook will explore the features of TFDV using the [Adult ](https://archive.ics.uci.edu/ml/datasets/Adult) dataset from UCI.

### Importing the data

Tensorflow works with datafrom Csv files or TFRecords, the dataset is composed by a Train and Eval dataset, the data is available in my Google Drive (Google colab has integration tools with Gdrive), but you can download at the link above in UCI.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [9]:
# Import libraries
from __future__ import print_function
import sys, os
import tempfile, urllib, zipfile

# Confirm it is using Python 2 due TFDV just works on python 2
assert sys.version_info.major is 2, 'Oops, not running Python 2'

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
OUTPUT_DIR = '/content/gdrive/My Drive/Export_Colab/'
TRAIN_DATA = os.path.join(BASE_DIR, 'train', 'data.csv')
EVAL_DATA  = os.path.join(BASE_DIR, 'eval', 'data.csv')

# Download the zip file from Google Drive and unzip it
zip = '/content/gdrive/My Drive/Datasets/Adult.zip'
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -lR {BASE_DIR}

Here's what we downloaded:
/tmp/tmpLeJsth:
total 16
-rw-r--r-- 1 root root 5229 Jan 17 12:13 adult.names
drwxr-xr-x 2 root root 4096 Jan 17 12:13 eval
drwxr-xr-x 2 root root 4096 Jan 17 12:13 train

/tmp/tmpLeJsth/eval:
total 1880
-rw-r--r-- 1 root root 1921726 Jan 17 12:13 data.csv

/tmp/tmpLeJsth/train:
total 3884
-rw-r--r-- 1 root root 3974305 Jan 17 12:13 data.csv


### TFDV

The TFDV is not available on the default packages in Google Colab, we install it  using pip

In [3]:
!pip install -q tensorflow_data_validation
import tensorflow_data_validation as tfdv

print('TFDV version: {}'.format(tfdv.version.__version__))

apache-beam 2.9.0 has requirement pytz<=2018.4,>=2018.3, but you'll have pytz 2018.9 which is incompatible.
TFDV version: 0.11.0


### Compute and visualize statistics

TFDV can compute descriptive statistics providing a quick overview of the data in terms of the features, shapes and distribution of the values.

First use tfdv.generate_statistics_from_csv to compute statistics for train data (Liander region), TFDV can compute descriptive statistics, generate_statistics_from_csv works specifying a file pattern. The pattern can contain glob characters  (*, ?, [...] and sets).

In [4]:
# Define column names
names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
         'marital_status', 'occupation', 'relationship', 'race', 'sex', 
         'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 
         'wage']

# Generate stats for train data
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA,
                                                column_names=names)

# Visualize
tfdv.visualize_statistics(train_stats)

it is possible to check the distribution of each variable (max, min, median, mean, std and zeros), how much missing data to each feature. The numeric and categorical variables are shown separated.

With this information it is possible have an idea about the data and infer some preliminary assumptios, most of the people work 30-40 hours per week, they are men married and receive less than 50k per year.

### Infer a schema

Infering a schema we save a lot of initial work, the scheme defines the data that are relevant to the model, for example, the type of each feature (numerical or categorical), for categorical features the schema also defines the domain - the list of acceptable values, TFDV provides a initial version of the schema based on the descriptive statistics.

Define the schema is important because the model pipeline relies on it also it is provide a documentation of the data.

In [5]:
# Define the schame
schema = tfdv.infer_schema(statistics=train_stats)

# Display the schema 
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'wage',STRING,required,,'wage'
'hours_per_week',INT,required,,-
'workclass',STRING,required,,'workclass'
'relationship',STRING,required,,'relationship'
'age',INT,required,,-
'marital_status',STRING,required,,'marital_status'
'sex',STRING,required,,'sex'
'race',STRING,required,,'race'
'education_num',INT,required,,-


,Values
Domain,
'wage',"' <=50K', ' >50K'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'sex',"' Female', ' Male'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia'"
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"


With the schema we have an overview how the schema was defined, looking at the domains we can note simbol "?", it represents the missing data, currently TFDV doesn't support input specify values as NaN, there is a open issue for this case [link](https://github.com/tensorflow/data-validation/issues/51), we will deal with this values in the next notebook with TFX.

### Check evaluation data for errors

We checked and infer a schema for the train dataset, the schema also should be consistent in the eval dataset, it should have the same range of values in the numerical features, if the data is different between the eval and train data we'll have problems in the model.

In [6]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA,
                                              column_names=names)

# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, 
                          rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', 
                          rhs_name='TRAIN_DATASET')

In evalation of the data TFDV shows in the plot both the train and eval stats overlaid, it is possible to check the distribuition visually, it can speedy the change in the schema when necessary.

Looking in the plots we can consider both the train and eval has the same distribution, the difenrrence in the plots is just in the quantity of each value in the features.

### Anomalies

After generate stats for both train and eval dataset, it necessary to check if the schema define to each one is the same (range of values, type of feature and so on), TFDV has a function to look for anomalies, e.g, features define different for each schema.

In [7]:
# Check eval data for errors by validating the Eval data stats using the 
# previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)

# Display anomalies
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'wage',Unexpected string values,"Examples contain values missing from the schema: <=50K. (~76%), >50K. (~23%)."


We have just one feature with anomalies, look deeply we can check the values are correct but the problem there is a period and a space in the values, so the TFDV considered it as a new values.

### Fixing the erros

The corrections will depend of our knowlegde of the data, looking at each feature and decide what action should be take, for our case it is not necessary to ajusted the schema, but to test how to correct the errors we will add the values to the domain and check again the anomalies

In [8]:
# Add new value to the domain of feature wage.
wage = tfdv.get_domain(schema, 'wage')
wage.value.append(' <=50K.')
wage.value.append(' >50K.')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

Running and check again the anomalies we found not any error

### Schema Environments

TFDV it is possible to define different schema according to your data, for example, you added labels in the train dataset but for prediction it is not necessary.

**Environments** can be used to express such requirements, features in schema can be associated with a set of environments using `default_environment`, `in_environment` and `not_in_environment`.

For our example we'll not use this function, since our train and eval data has the same schema.

### Drift and Skew

TFDV also provides additional functionalities to detect drift and skew comparing the statistics of the different datasets, for Skew analysis TFDV has support to three kind (Schema Skew, Feature Skew and Distribuition Skew)

This feature are used in advanced analysis and are not focus of the notebook, for more information please visite [TFDV](https://www.tensorflow.org/tfx/data_validation/) website.

### Freeze Schema

After all ajusts is time to save for future use.

In [19]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'adult_schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {'/content/gdrive/"My Drive"/Export_Colab/adult_schema.pbtxt'}

feature {
  name: "wage"
  type: BYTES
  domain: "wage"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "hours_per_week"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "workclass"
  type: BYTES
  domain: "workclass"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "relationship"
  type: BYTES
  domain: "relationship"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "age"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "marital_status"
  type: BYTES
  domain: "marital_status"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sex"
  type: BYT

### Conclusion

TFDV deliveres what it is promissed, it shows a summary of your data and helps to define the schema for using in TFX, when working with several files and data, it takes time to undestand all the data and make assumptions, after you spend more time defining what to do for each feature.

You can refine your model using TDFV, all your features are visually in front you, it's easy to undestand the changes. TFDV helps in this task let it more clear and inside the tensforflow ecosystem. It was a great addition by Google, for more information and new releases in TFDV please check Tensorflow [site](https://www.tensorflow.org/tfx/data_validation/).

In the next notebooks I'll cover the others TFX features (Transform, Model Analysis, Serving and API)